In [84]:
import torch

class Dummy:
    pass
args = Dummy
args.spec = "test_cases/fc1/img0_0.06000.txt"

with open(args.spec, 'r') as f:
    lines = [line[:-1] for line in f.readlines()]
    true_label = int(lines[0])
    pixel_values = [float(line) for line in lines[1:]]
    eps = float(args.spec[:-4].split('/')[-1].split('_')[-1])
    
DEVICE = 'cpu'
INPUT_SIZE = 28
inp = torch.FloatTensor(pixel_values).view(1, 1, INPUT_SIZE, INPUT_SIZE).to(DEVICE)

inp.shape

torch.Size([1, 1, 28, 28])

In [85]:
eps = 0.35

inp = torch.rand(1, 1, 5, 5)
inp

tensor([[[[0.5790, 0.6730, 0.5082, 0.1987, 0.4940],
          [0.4158, 0.8500, 0.8630, 0.9614, 0.6980],
          [0.3972, 0.2495, 0.8038, 0.4250, 0.8320],
          [0.9373, 0.8173, 0.3423, 0.1890, 0.3476],
          [0.3623, 0.8999, 0.4661, 0.1991, 0.8134]]]])

In [88]:
upper = inp + eps
lower = inp - eps
upper.clamp_(max=1) # clip input_zonotope to the input space
lower.clamp_(min=0)
a0 = (upper + lower) / 2 # center of the zonotope

print(upper)
print(lower)
print(a0.shape)
print(a0[0][0].shape)

tensor([[[[0.9290, 1.0000, 0.8582, 0.5487, 0.8440],
          [0.7658, 1.0000, 1.0000, 1.0000, 1.0000],
          [0.7472, 0.5995, 1.0000, 0.7750, 1.0000],
          [1.0000, 1.0000, 0.6923, 0.5390, 0.6976],
          [0.7123, 1.0000, 0.8161, 0.5491, 1.0000]]]])
tensor([[[[0.2290, 0.3230, 0.1582, 0.0000, 0.1440],
          [0.0658, 0.5000, 0.5130, 0.6114, 0.3480],
          [0.0472, 0.0000, 0.4538, 0.0750, 0.4820],
          [0.5873, 0.4673, 0.0000, 0.0000, 0.0000],
          [0.0123, 0.5499, 0.1161, 0.0000, 0.4634]]]])
torch.Size([1, 1, 5, 5])
torch.Size([5, 5])


In [87]:
# A must have shape (nb_error_terms, *[shape of input])
# for the input layer, there is 1 error term for each pixel, so nb_error_terms = inp.numel()
A = torch.zeros(a0.numel(), *a0[0].shape)
a0[0][0] == a0[0][0]
# A[:, a0[0]].shape
# torch.diag(((upper - lower) / 2).reshape(-1)).shape

# A[:, torch.ones(1, 5, 5, dtype=torch.bool)]
# torch.ones(1, 5, 5, dtype=torch.bool)
# = torch.diag(((upper - lower) / 2).reshape(-1))
# A

mask = torch.ones(1, 5, 5, dtype=torch.bool)
A[:, mask] = torch.diag( ((upper - lower) / 2).reshape(-1) )
A

tensor([[[[0.3500, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.3385, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.3500, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000, 0.2744, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
